# call_slide_extra: exp22ベースの予測・シミュレーション（ExtraTrees版）

**ベース**: exp22（冗長な特徴量の削減 + Optuna再最適化）

**モデル**: ExtraTrees（exp22で最適化されたパラメータ）

**追加機能**:
1. holdout期間の予測CSV出力（pred_model.csv）
2. 差分シミュレーションで月別の回避額を計算（sim_daily.csv, sim_monthly.csv）
3. 標準出力に要約表示（MAE/WAPE、savings）

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import warnings
warnings.filterwarnings('ignore')
import os

# 出力ディレクトリ
output_dir = '../output/call_slide_extra'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("出力ディレクトリ:", output_dir)

出力ディレクトリ: ../output/call_slide_extra


---
## 1. データの読み込みと特徴量作成（exp22と同じ）

In [2]:
# ==================================================================================
# データの読み込みと特徴量作成（exp22と同じ）
# ==================================================================================

def load_and_preprocess_data():
    calender = pd.read_csv('../input/calender_data.csv')
    cm_data = pd.read_csv('../input/cm_data.csv')
    gt_service = pd.read_csv('../input/gt_service_name.csv')
    acc_get = pd.read_csv('../input/regi_acc_get_data_transform.csv')
    call_data = pd.read_csv('../input/regi_call_data_transform.csv')
    
    calender['cdr_date'] = pd.to_datetime(calender['cdr_date'])
    cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
    acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
    call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])
    gt_service['week'] = pd.to_datetime(gt_service['week'])
    
    return calender, cm_data, gt_service, acc_get, call_data

def merge_datasets(calender, cm_data, gt_service, acc_get, call_data):
    df = call_data.copy()
    df = df.merge(calender, on='cdr_date', how='left')
    df = df.merge(cm_data, on='cdr_date', how='left')
    df = df.merge(acc_get, on='cdr_date', how='left')
    
    gt_service_daily = []
    for idx, row in gt_service.iterrows():
        week_start = row['week']
        for i in range(7):
            date = week_start + timedelta(days=i)
            gt_service_daily.append({'cdr_date': date, 'search_cnt': row['search_cnt']})
    
    gt_daily = pd.DataFrame(gt_service_daily)
    df = df.merge(gt_daily, on='cdr_date', how='left')
    
    return df

def create_basic_time_features(df):
    df = df.copy()
    df['year'] = df['cdr_date'].dt.year
    df['month'] = df['cdr_date'].dt.month
    df['day_of_month'] = df['cdr_date'].dt.day
    df['quarter'] = df['cdr_date'].dt.quarter
    df['day_of_year'] = df['cdr_date'].dt.dayofyear
    df['week_of_year'] = df['cdr_date'].dt.isocalendar().week
    df['days_from_start'] = (df['cdr_date'] - df['cdr_date'].min()).dt.days
    df['is_month_start'] = (df['day_of_month'] <= 5).astype(int)
    df['is_month_end'] = (df['day_of_month'] >= 25).astype(int)
    return df

def create_lag_features(df, target_col='call_num', lags=[1, 2, 3, 5, 7, 14, 30]):
    df = df.copy()
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    return df

def create_rolling_features(df, target_col='call_num', windows=[3, 7, 14, 30]):
    df = df.copy()
    for window in windows:
        df[f'ma_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).mean()
        df[f'ma_std_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).std()
    return df

def create_aggregated_features(df):
    df = df.copy()
    df['cm_7d'] = df['cm_flg'].shift(1).rolling(window=7, min_periods=1).sum()
    df['gt_ma_7'] = df['search_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
    df['acc_ma_7'] = df['acc_get_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
    
    df['dow_avg'] = np.nan
    for dow in df['dow'].unique():
        mask = df['dow'] == dow
        df.loc[mask, 'dow_avg'] = df.loc[mask, 'call_num'].shift(1).expanding().mean()
    
    return df

def create_acc_get_features(df):
    df = df.copy()
    df['acc_get_lag7'] = df['acc_get_cnt'].shift(7)
    df['acc_get_sum_14d'] = df['acc_get_cnt'].shift(1).rolling(window=14, min_periods=1).sum()
    return df

def create_regime_change_features(df):
    df = df.copy()
    
    tax_implementation_date = pd.Timestamp('2019-10-01')
    rush_deadline = pd.Timestamp('2019-09-30')
    
    df['days_to_2019_10_01'] = (tax_implementation_date - df['cdr_date']).dt.days
    df['is_pre_2019_10_01'] = (df['cdr_date'] < tax_implementation_date).astype(int)
    df['is_post_2019_10_01'] = (df['cdr_date'] >= tax_implementation_date).astype(int)
    
    df['days_to_2019_09_30'] = (rush_deadline - df['cdr_date']).dt.days
    df['is_pre_2019_09_30'] = (df['cdr_date'] < rush_deadline).astype(int)
    df['is_post_2019_09_30'] = (df['cdr_date'] >= rush_deadline).astype(int)
    
    rush_start = rush_deadline - pd.Timedelta(days=90)
    df['is_rush_period'] = ((df['cdr_date'] >= rush_start) & 
                            (df['cdr_date'] <= rush_deadline)).astype(int)
    
    adaptation_end = tax_implementation_date + pd.Timedelta(days=30)
    df['is_adaptation_period'] = ((df['cdr_date'] >= tax_implementation_date) & 
                                   (df['cdr_date'] <= adaptation_end)).astype(int)
    
    return df

print('データ読み込み・特徴量作成関数を定義しました')

データ読み込み・特徴量作成関数を定義しました


In [3]:
# ==================================================================================
# データ準備
# ==================================================================================

print("*" * 80)
print("call_slide_extra: exp22ベースの予測・シミュレーション（ExtraTrees版）")
print("*" * 80)

calender, cm_data, gt_service, acc_get, call_data = load_and_preprocess_data()
df = merge_datasets(calender, cm_data, gt_service, acc_get, call_data)
df = create_basic_time_features(df)
df = create_lag_features(df)
df = create_rolling_features(df)
df = create_aggregated_features(df)
df = create_acc_get_features(df)
df = create_regime_change_features(df)

# 翌日の入電数を目的変数にする
# target_next_day = call_num.shift(-1) は「今日のXで、明日のYを予測」
# つまり、cdr_date=D の行で target_next_day は D+1 の call_num
df['target_next_day'] = df['call_num'].shift(-1)
df = df.dropna(subset=['target_next_day']).reset_index(drop=True)

# 平日のみ
df_model = df[df['dow'].isin([1, 2, 3, 4, 5])].copy().reset_index(drop=True)

print(f"\n平日データ数: {len(df_model)}行")
print(f"期間: {df_model['cdr_date'].min()} ~ {df_model['cdr_date'].max()}")

********************************************************************************
call_slide_extra: exp22ベースの予測・シミュレーション（ExtraTrees版）
********************************************************************************

平日データ数: 477行
期間: 2018-06-01 00:00:00 ~ 2020-03-30 00:00:00


In [4]:
# ==================================================================================
# exp22の特徴量（37個）
# ==================================================================================

feature_cols_exp22 = [
    # 基本時系列特徴量
    'dow', 'day_of_month', 'year', 
    'day_of_year', 'week_of_year',
    'is_month_start', 'is_month_end',
    # カレンダー特徴量
    'day_before_holiday_flag',
    # 外部データ
    'cm_flg', 'acc_get_cnt', 'search_cnt',
    # 集約特徴量
    'cm_7d', 'gt_ma_7', 'acc_ma_7', 'dow_avg',
    # ラグ特徴量
    'lag_1', 'lag_2', 'lag_3', 'lag_5', 'lag_7', 'lag_14', 'lag_30',
    # 移動平均特徴量（ma_14, ma_std_14削除）
    'ma_3', 'ma_7', 'ma_30',
    'ma_std_3', 'ma_std_7', 'ma_std_30',
    # レジーム変化特徴量（is_pre_*削除、days_to_2019_09_30削除）
    'days_to_2019_10_01', 'is_post_2019_10_01',
    'is_post_2019_09_30',
    'is_rush_period', 'is_adaptation_period',
]

print(f"特徴量数: {len(feature_cols_exp22)}")

特徴量数: 33


In [5]:
# ==================================================================================
# Holdout Validation 設定（exp22と同じ）
# ==================================================================================

# 欠損値を除去
df_clean = df_model.dropna(subset=feature_cols_exp22 + ['target_next_day']).copy()

print("\n" + "=" * 80)
print("Holdout Validation 設定")
print("=" * 80)

# Holdout分割
test_start_date = pd.Timestamp('2020-01-30')
train_end_date = test_start_date - pd.Timedelta(days=1)

train_df = df_clean[df_clean['cdr_date'] <= train_end_date].copy()
test_df = df_clean[df_clean['cdr_date'] >= test_start_date].copy()

X_train = train_df[feature_cols_exp22]
y_train = train_df['target_next_day']
X_test = test_df[feature_cols_exp22]
y_test = test_df['target_next_day']

print(f"\nTrain: {len(X_train)}件 ({train_df['cdr_date'].min().strftime('%Y-%m-%d')} ~ {train_df['cdr_date'].max().strftime('%Y-%m-%d')})")
print(f"Test : {len(X_test)}件 ({test_df['cdr_date'].min().strftime('%Y-%m-%d')} ~ {test_df['cdr_date'].max().strftime('%Y-%m-%d')})")


Holdout Validation 設定

Train: 413件 (2018-07-02 ~ 2020-01-29)
Test : 43件 (2020-01-30 ~ 2020-03-30)


In [6]:
# ==================================================================================
# 評価関数
# ==================================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_wape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100

def evaluate_model(y_true, y_pred):
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred),
        'WAPE': calculate_wape(y_true, y_pred)
    }

print('評価関数を定義しました')

評価関数を定義しました


---
## 2. モデル学習（exp22のExtraTreesパラメータを使用）

In [7]:
# ==================================================================================
# exp22で最適化されたExtraTreesパラメータ
# ==================================================================================

from sklearn.ensemble import ExtraTreesRegressor

# exp22のOptuna最適化パラメータ（exp19ベースライン）
# exp22のセル12でOptuna最適化が行われるが、ベースラインとしてexp19のパラメータを使用
EXTRATREES_PARAMS = {
    'n_estimators': 472,
    'max_depth': 35,
    'min_samples_split': 15,
    'min_samples_leaf': 4,
    'max_features': None
}

print("\n" + "=" * 80)
print("モデル学習（ExtraTrees）")
print("=" * 80)
print("\nパラメータ:")
for key, value in EXTRATREES_PARAMS.items():
    print(f"  {key}: {value}")

model = ExtraTreesRegressor(**EXTRATREES_PARAMS, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Holdout予測
y_pred_model = model.predict(X_test)

# 評価
model_metrics = evaluate_model(y_test, y_pred_model)
print(f"\nExtraTrees Holdout 評価:")
print(f"  MAE:  {model_metrics['MAE']:.2f}")
print(f"  RMSE: {model_metrics['RMSE']:.2f}")
print(f"  R2:   {model_metrics['R2']:.4f}")
print(f"  WAPE: {model_metrics['WAPE']:.2f}%")


モデル学習（ExtraTrees）

パラメータ:
  n_estimators: 472
  max_depth: 35
  min_samples_split: 15
  min_samples_leaf: 4
  max_features: None

ExtraTrees Holdout 評価:
  MAE:  16.00
  RMSE: 21.43
  R2:   0.8682
  WAPE: 16.68%


---
## 3. 追加要件1: pred_model.csv の出力

**日付整合の確認**:
- exp22では `target_next_day = call_num.shift(-1)` で翌日予測
- つまり、cdr_date=D の行の target_next_day は D+1 の call_num
- したがって、予測 pred_model も D+1 の入電数を予測している
- actual_call_num と pred_model を同じ日付に揃えるには、pred_model の日付を +1日 shift する必要がある

In [8]:
# ==================================================================================
# 追加要件1: pred_model.csv の出力
# ==================================================================================

print("\n" + "=" * 80)
print("追加要件1: pred_model.csv の出力")
print("=" * 80)

# test_df の cdr_date は「予測を行った日」
# target_next_day は「翌日の実績」= 予測対象日の実績
# したがって、予測対象日 = cdr_date + 1日

pred_df = pd.DataFrame({
    'cdr_date_input': test_df['cdr_date'].values,  # 予測を行った日（特徴量の日付）
    'actual_call_num': y_test.values,               # 実際の入電数（翌日の実績）
    'pred_model': y_pred_model,                     # モデル予測（翌日の予測）
})

# 予測対象日（実績の日付）に揃える
pred_df['cdr_date'] = pred_df['cdr_date_input'] + pd.Timedelta(days=1)

# 営業日（平日）でフィルタ（土日を除外）
pred_df['dow'] = pred_df['cdr_date'].dt.dayofweek + 1  # 1=月曜, 7=日曜
pred_df = pred_df[pred_df['dow'].isin([1, 2, 3, 4, 5])].copy()

print(f"\n日付整合の確認:")
print(f"  予測入力日（特徴量日）: {pred_df['cdr_date_input'].min()} ~ {pred_df['cdr_date_input'].max()}")
print(f"  予測対象日（実績日）  : {pred_df['cdr_date'].min()} ~ {pred_df['cdr_date'].max()}")


追加要件1: pred_model.csv の出力

日付整合の確認:
  予測入力日（特徴量日）: 2020-01-30 00:00:00 ~ 2020-03-30 00:00:00
  予測対象日（実績日）  : 2020-01-31 00:00:00 ~ 2020-03-31 00:00:00


In [9]:
# ==================================================================================
# pred_baseline（前週同曜日）の作成
# ==================================================================================

# 前週同曜日 = 7営業日前ではなく、7カレンダー日前の同じ曜日
# 平日のみのデータなので、7営業日前を計算する

# 全期間の平日データから call_num を取得
all_weekday_df = df_model[['cdr_date', 'call_num']].copy()
all_weekday_df = all_weekday_df.sort_values('cdr_date').reset_index(drop=True)

# 5営業日前の値を取得（5営業日 = 1週間前の同曜日）
all_weekday_df['call_num_7bd_ago'] = all_weekday_df['call_num'].shift(5)

# pred_df に結合
pred_df = pred_df.merge(
    all_weekday_df[['cdr_date', 'call_num_7bd_ago']],
    on='cdr_date',
    how='left'
)
pred_df['pred_baseline'] = pred_df['call_num_7bd_ago']

# 欠損を除外
pred_df_clean = pred_df.dropna(subset=['pred_baseline']).copy()

print(f"\npred_baseline（前週同曜日）を作成:")
print(f"  有効データ数: {len(pred_df_clean)}件（欠損除外後）")


pred_baseline（前週同曜日）を作成:
  有効データ数: 33件（欠損除外後）


In [10]:
# ==================================================================================
# pred_model.csv の出力
# ==================================================================================

pred_output = pred_df_clean[['cdr_date', 'actual_call_num', 'pred_model', 'pred_baseline']].copy()
pred_output['cdr_date'] = pred_output['cdr_date'].dt.strftime('%Y-%m-%d')

pred_output.to_csv(f'{output_dir}/pred_model.csv', index=False)
print(f"\n保存しました: {output_dir}/pred_model.csv")
print(f"\nサンプル（先頭5行）:")
print(pred_output.head().to_string(index=False))


保存しました: ../output/call_slide_extra/pred_model.csv

サンプル（先頭5行）:
  cdr_date  actual_call_num  pred_model  pred_baseline
2020-01-31            125.0  137.279777          121.0
2020-02-04            116.0  144.278439          126.0
2020-02-05            153.0  143.125390          156.0
2020-02-06            135.0  140.924742          156.0
2020-02-07            147.0  137.324032          125.0


---
## 4. 追加要件2: 差分シミュレーション

**定義**:
- cap = prediction * (1 + buffer)
- overflow = max(0, actual_call_num - cap)
- cost = overflow * CPC

**パラメータ**: buffer=0.10、CPC=600/800円

In [11]:
# ==================================================================================
# 追加要件2: 差分シミュレーション
# ==================================================================================

print("\n" + "=" * 80)
print("追加要件2: 差分シミュレーション")
print("=" * 80)

# パラメータ
BUFFER = 0.10
CPC_600 = 600
CPC_800 = 800

print(f"\nパラメータ:")
print(f"  buffer: {BUFFER*100:.0f}%")
print(f"  CPC: {CPC_600}円, {CPC_800}円")

# シミュレーション用データ
sim_df = pred_df_clean.copy()

# cap の計算
sim_df['cap_baseline'] = sim_df['pred_baseline'] * (1 + BUFFER)
sim_df['cap_model'] = sim_df['pred_model'] * (1 + BUFFER)

# overflow の計算
sim_df['overflow_baseline'] = np.maximum(0, sim_df['actual_call_num'] - sim_df['cap_baseline'])
sim_df['overflow_model'] = np.maximum(0, sim_df['actual_call_num'] - sim_df['cap_model'])

# cost の計算
sim_df['cost_baseline_600'] = sim_df['overflow_baseline'] * CPC_600
sim_df['cost_model_600'] = sim_df['overflow_model'] * CPC_600
sim_df['cost_baseline_800'] = sim_df['overflow_baseline'] * CPC_800
sim_df['cost_model_800'] = sim_df['overflow_model'] * CPC_800

print(f"\nシミュレーション対象データ数: {len(sim_df)}件")


追加要件2: 差分シミュレーション

パラメータ:
  buffer: 10%
  CPC: 600円, 800円

シミュレーション対象データ数: 33件


In [12]:
# ==================================================================================
# sim_daily.csv の出力
# ==================================================================================

sim_daily_output = sim_df[[
    'cdr_date', 'actual_call_num', 'pred_baseline', 'pred_model',
    'cap_baseline', 'cap_model', 'overflow_baseline', 'overflow_model',
    'cost_baseline_600', 'cost_model_600', 'cost_baseline_800', 'cost_model_800'
]].copy()

sim_daily_output['cdr_date'] = pd.to_datetime(sim_daily_output['cdr_date']).dt.strftime('%Y-%m-%d')

sim_daily_output.to_csv(f'{output_dir}/sim_daily.csv', index=False)
print(f"保存しました: {output_dir}/sim_daily.csv")
print(f"\nサンプル（先頭5行）:")
print(sim_daily_output.head().to_string(index=False))

保存しました: ../output/call_slide_extra/sim_daily.csv

サンプル（先頭5行）:
  cdr_date  actual_call_num  pred_baseline  pred_model  cap_baseline  cap_model  overflow_baseline  overflow_model  cost_baseline_600  cost_model_600  cost_baseline_800  cost_model_800
2020-01-31            125.0          121.0  137.279777         133.1 151.007754                0.0             0.0                0.0             0.0                0.0             0.0
2020-02-04            116.0          126.0  144.278439         138.6 158.706283                0.0             0.0                0.0             0.0                0.0             0.0
2020-02-05            153.0          156.0  143.125390         171.6 157.437929                0.0             0.0                0.0             0.0                0.0             0.0
2020-02-06            135.0          156.0  140.924742         171.6 155.017216                0.0             0.0                0.0             0.0                0.0             0.0
2020-02-07   

In [13]:
# ==================================================================================
# sim_monthly.csv の出力（月別集計）
# ==================================================================================

sim_df['year_month'] = pd.to_datetime(sim_df['cdr_date']).dt.to_period('M').astype(str)

monthly_agg = sim_df.groupby('year_month').agg({
    'cost_baseline_600': 'sum',
    'cost_model_600': 'sum',
    'cost_baseline_800': 'sum',
    'cost_model_800': 'sum',
}).reset_index()

# savings の計算
monthly_agg['savings_600'] = monthly_agg['cost_baseline_600'] - monthly_agg['cost_model_600']
monthly_agg['savings_800'] = monthly_agg['cost_baseline_800'] - monthly_agg['cost_model_800']

# 列の順序を整理
sim_monthly_output = monthly_agg[[
    'year_month', 'savings_600', 'savings_800',
    'cost_baseline_600', 'cost_model_600',
    'cost_baseline_800', 'cost_model_800'
]].copy()

# 列名をわかりやすく変更
sim_monthly_output.columns = [
    'year_month', 'savings_600', 'savings_800',
    'baseline_cost_600', 'model_cost_600',
    'baseline_cost_800', 'model_cost_800'
]

sim_monthly_output.to_csv(f'{output_dir}/sim_monthly.csv', index=False)
print(f"保存しました: {output_dir}/sim_monthly.csv")
print(f"\n月別集計:")
print(sim_monthly_output.to_string(index=False))

保存しました: ../output/call_slide_extra/sim_monthly.csv

月別集計:
year_month   savings_600   savings_800  baseline_cost_600  model_cost_600  baseline_cost_800  model_cost_800
   2020-01      0.000000      0.000000                0.0        0.000000                0.0        0.000000
   2020-02 117814.935773 157086.581031           160800.0    42985.064227           214400.0    57313.418969
   2020-03 106036.796156 141382.394875           118860.0    12823.203844           158480.0    17097.605125


---
## 5. 追加要件3: 標準出力に要約表示

In [14]:
# ==================================================================================
# 追加要件3: 標準出力に要約表示
# ==================================================================================

print("\n" + "=" * 80)
print("要約")
print("=" * 80)

# Holdout MAE/WAPE
print("\n【Holdout 評価指標】")
print(f"  モデル（ExtraTrees）:")
print(f"    MAE:  {model_metrics['MAE']:.2f}")
print(f"    WAPE: {model_metrics['WAPE']:.2f}%")

# Baselineの評価
baseline_mae = mean_absolute_error(pred_df_clean['actual_call_num'], pred_df_clean['pred_baseline'])
baseline_wape = calculate_wape(pred_df_clean['actual_call_num'], pred_df_clean['pred_baseline'])
print(f"  ベースライン（前週同曜日）:")
print(f"    MAE:  {baseline_mae:.2f}")
print(f"    WAPE: {baseline_wape:.2f}%")

# 差分シミュレーション合計
total_savings_600 = sim_monthly_output['savings_600'].sum()
total_savings_800 = sim_monthly_output['savings_800'].sum()
total_baseline_cost_600 = sim_monthly_output['baseline_cost_600'].sum()
total_baseline_cost_800 = sim_monthly_output['baseline_cost_800'].sum()

print("\n【差分シミュレーション合計】")
print(f"  CPC=600円:")
print(f"    合計節約額: {total_savings_600:,.0f}円")
print(f"    ベースラインコスト: {total_baseline_cost_600:,.0f}円")
print(f"    削減率: {total_savings_600/total_baseline_cost_600*100:.1f}%" if total_baseline_cost_600 > 0 else "    削減率: N/A")

print(f"  CPC=800円:")
print(f"    合計節約額: {total_savings_800:,.0f}円")
print(f"    ベースラインコスト: {total_baseline_cost_800:,.0f}円")
print(f"    削減率: {total_savings_800/total_baseline_cost_800*100:.1f}%" if total_baseline_cost_800 > 0 else "    削減率: N/A")

# 上位の月（節約額が大きい月）
print("\n【節約額上位の月】")
top_months = sim_monthly_output.sort_values('savings_600', ascending=False).head(3)
print("  CPC=600円:")
for _, row in top_months.iterrows():
    print(f"    {row['year_month']}: {row['savings_600']:,.0f}円")

top_months_800 = sim_monthly_output.sort_values('savings_800', ascending=False).head(3)
print("  CPC=800円:")
for _, row in top_months_800.iterrows():
    print(f"    {row['year_month']}: {row['savings_800']:,.0f}円")


要約

【Holdout 評価指標】
  モデル（ExtraTrees）:
    MAE:  16.00
    WAPE: 16.68%
  ベースライン（前週同曜日）:
    MAE:  34.91
    WAPE: 28.63%

【差分シミュレーション合計】
  CPC=600円:
    合計節約額: 223,852円
    ベースラインコスト: 279,660円
    削減率: 80.0%
  CPC=800円:
    合計節約額: 298,469円
    ベースラインコスト: 372,880円
    削減率: 80.0%

【節約額上位の月】
  CPC=600円:
    2020-02: 117,815円
    2020-03: 106,037円
    2020-01: 0円
  CPC=800円:
    2020-02: 157,087円
    2020-03: 141,382円
    2020-01: 0円


In [15]:
# ==================================================================================
# 出力ファイル一覧
# ==================================================================================

print("\n" + "=" * 80)
print("出力ファイル一覧")
print("=" * 80)
print(f"  1. {output_dir}/pred_model.csv")
print(f"  2. {output_dir}/sim_daily.csv")
print(f"  3. {output_dir}/sim_monthly.csv")
print("\n完了")


出力ファイル一覧
  1. ../output/call_slide_extra/pred_model.csv
  2. ../output/call_slide_extra/sim_daily.csv
  3. ../output/call_slide_extra/sim_monthly.csv

完了
